In [28]:
! pip install haystack-ai haystack-integrations

Der Befehl "pip" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [29]:
import pandas as pd
import json
import re
from typing import List, Dict, Any, Optional
from haystack import Pipeline, Document, component
from haystack.components.builders import ChatPromptBuilder
from haystack.components.retrievers import InMemoryBM25Retriever
from haystack.document_stores.in_memory import InMemoryDocumentStore 
from haystack.dataclasses import ChatMessage
# Import OllamaChatGenerator - muss separat installiert werden
from haystack_integrations.components.generators.ollama import OllamaChatGenerator

In [30]:
#3a 


bpmn_examples = [
    {"bpmn": '<bpmn:startEvent id="StartEvent_1"/><bpmn:task id="T1" name="Check Credit"/><bpmn:task id="T2" name="Approve Loan"/>', 
     "text": "The process starts with checking the customer's credit score. If the score is sufficient, the task is to approve the loan."},
    {"bpmn": '<bpmn:startEvent id="Start"/><bpmn:task id="T3" name="Process Order"/><bpmn:task id="T4" name="Ship Product"/><bpmn:endEvent id="End"/>', 
     "text": "First, process the received order. Once processing is complete, ship the final product to the customer."}
]


In [31]:
#3a

document_store = InMemoryDocumentStore()
documents = [
    Document(content=ex["bpmn"], meta={"text": ex["text"], "id": f"doc_{i}"})
    for i, ex in enumerate(bpmn_examples)
]
document_store.write_documents(documents)

retriever = InMemoryBM25Retriever(document_store=document_store)

In [32]:
prompt_template = [
    ChatMessage.from_system("""
You are an expert BPMN text generator. Your task is to analyze the user-provided 
BPMN XML fragments and create a clear, natural language description of the process.

CRITICAL RULE: The tasks in your description MUST be enclosed in <bpmn:task>Task Name</bpmn:task> tags.
Example Tag Usage: <bpmn:task>Task Name</bpmn:task>
    """),
    ChatMessage.from_user("""
--- Example Context (Retrieved from Document Store) ---
BPMN Example XML: {{examples.0.content}}
Text Example: {{examples.0.meta.text}}
------------------------------------------------------

Now, create a textual description for the following BPMN XML fragment:
{{query_bpmn}}

Remember to use the <bpmn:task>Task Name</bpmn:task> tags for every task name.
    """)
]
prompt_builder = ChatPromptBuilder(template=prompt_template, required_variables=["query_bpmn", "examples"])


In [33]:


# 2c. Matching Generator: LLM to perform the generation.
# NOTE: Replace the model and URL if necessary.
chat_generator = OllamaChatGenerator(
    model="llama3.1:8b",
    url="http://localhost:11434",
    timeout=30*60,
    generation_kwargs={"temperature": 0.3}
)

In [34]:
# Pipeline, that recieves all comoponents
bpmn_pipeline = Pipeline()

# Add components
bpmn_pipeline.add_component(instance=retriever, name="retriever")
bpmn_pipeline.add_component(instance=prompt_builder, name="prompt_builder")
bpmn_pipeline.add_component(instance=chat_generator, name="generator")

# Connect components
# 1. Provide the query (the new BPMN model) to the Retriever
bpmn_pipeline.connect("retriever.documents", "prompt_builder.examples") 
# 2. Pass the retrieved documents (as 'examples') to the PromptBuilder
bpmn_pipeline.connect("prompt_builder.prompt", "generator.messages")

🚅 Components
  - retriever: InMemoryBM25Retriever
  - prompt_builder: ChatPromptBuilder
  - generator: OllamaChatGenerator
🛤️ Connections
  - retriever.documents -> prompt_builder.examples (list[Document])
  - prompt_builder.prompt -> generator.messages (list[ChatMessage])

In [35]:
# Test BPMN model with matching generator 3a 2.
test_bpmn = """<bpmn:startEvent id="Start"/>
<bpmn:task id="T1" name="Log Complaint"/>
<bpmn:task id="T2" name="Investigate Issue"/>
<bpmn:task id="T3" name="Process Refund"/>"""

# Run pipeline
result = bpmn_pipeline.run({
    "retriever": {"query": test_bpmn, "top_k": 1},
    "prompt_builder": {"query_bpmn": test_bpmn}
})

print("Generated annotated text:")
print(result["generator"]["replies"][0].text)


Generated annotated text:
The process begins with a start event. Next, log the customer's complaint in detail. After that, investigate the issue thoroughly. Finally, process the refund request.


In [36]:
import pandas as pd
import re
from typing import List, Dict

# 1. Define 2 test BPMN models with ground truth (Exercise 3b requirement)
TEST_BPMN_MODELS = [
    {
        "name": "Customer Onboarding (5 Tasks)",
        "xml": """<bpmn:startEvent id="Start"/>
<bpmn:task id="T1" name="Verify Identity"/>
<bpmn:task id="T2" name="Perform KYC Check"/>
<bpmn:task id="T3" name="Set up Account"/>
<bpmn:task id="T4" name="Approve Application"/>
<bpmn:task id="T5" name="Fund Account"/>""",
        "ground_truth": ["Verify Identity", "Perform KYC Check", "Set up Account", "Approve Application", "Fund Account"]
    },
    {
        "name": "Complaint Resolution (6 Tasks)",
        "xml": """<bpmn:startEvent id="Start"/>
<bpmn:task id="T1" name="Log Complaint"/>
<bpmn:task id="T2" name="Investigate Issue"/>
<bpmn:task id="T3" name="Process Refund"/>
<bpmn:task id="T4" name="Notify Finance"/>
<bpmn:task id="T5" name="Send Resolution Email"/>
<bpmn:task id="T6" name="Update CRM"/>""",
        "ground_truth": ["Log Complaint", "Investigate Issue", "Process Refund", "Notify Finance", "Send Resolution Email", "Update CRM"]
    }
]

# 2. Task extraction from annotated LLM output
def extract_and_clean_tasks(text: str) -> List[str]:
    """Extract task names from <bpmn:task> tags - FIXED REGEX"""
    tasks = re.findall(r'<bpmn:task[^>]*name="([^"]*)"[^>]*>', text, re.IGNORECASE)
    return [task.strip() for task in tasks if task.strip()]

# 3. Precision/Recall calculation (standard IR metrics for BPMN task extraction)
def calculate_precision_recall(predicted: List[str], ground_truth: List[str]) -> Dict:
    pred_set = {t.lower() for t in predicted if t}
    gt_set = {t.lower() for t in ground_truth}
    
    tp = len(pred_set & gt_set)
    fp = len(pred_set - gt_set)
    fn = len(gt_set - pred_set)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    
    return {
        "Precision": round(precision, 4),
        "Recall": round(recall, 4),
        "TP": tp, "FP": fp, "FN": fn,
        "GT_Total": len(ground_truth)
    }

# 4. Run pipeline for 2 models + evaluate
print("=== EXERCISE 3b) PIPELINE EXECUTION & EVALUATION ===\n")
results_summary = []

for i, model in enumerate(TEST_BPMN_MODELS, 1):
    print(f"{'='*60}")
    print(f"Model {i}: {model['name']}")
    print(f"{'='*60}")
    
    # Run the pipeline (assumes bpmn_annotation_pipeline from 3a exists)
    try:
        result = bpmn_pipeline.run({
            "retriever": {"query": model["xml"], "top_k": 1},
            "promptbuilder": {"query_bpmn": model["xml"]}
        })
        
        llm_output = result["generator"]["replies"][0].text
        predicted_tasks = extract_bpmn_tasks(llm_output)
        
        print("✓ Pipeline executed successfully")
        print("LLM Output preview:", llm_output[:300] + "..." if len(llm_output) > 300 else llm_output)
        
    except Exception as e:
        print(f"✗ Pipeline failed: {e}")
        predicted_tasks = []
        llm_output = "ERROR"
    
    # Calculate scores
    scores = calculate_precision_recall(predicted_tasks, model["ground_truth"])
    
    # Store for presentation
    results_summary.append({
        "Model": model["name"],
        "GroundTruth": model["ground_truth"],
        "PredictedTasks": predicted_tasks,
        "LLM_Output": llm_output,
        "Scores": scores
    })
    
    # Print detailed results
    print(f"\n📊 EVALUATION RESULTS:")
    print(f"  Ground Truth ({len(model['ground_truth'])} tasks): {model['ground_truth']}")
    print(f"  Extracted Tasks ({len(predicted_tasks)}): {predicted_tasks}")
    print(f"  Precision: {scores['Precision']:.4f} | Recall: {scores['Recall']:.4f}")
    print(f"  TP: {scores['TP']} | FP: {scores['FP']} | FN: {scores['FN']}")
    print()

# 5. Presentation-ready table (copy to slides)
print("\n" + "="*80)
print("📈 PRESENTATION TABLE - Exercise 3b Results")
print("="*80)

scores_df = pd.DataFrame([
    {
        "Model": r["Model"],
        "Precision": r["Scores"]["Precision"],
        "Recall": r["Scores"]["Recall"],
        "TP": r["Scores"]["TP"],
        "FP": r["Scores"]["FP"], 
        "FN": r["Scores"]["FN"],
        "GT_Total": r["Scores"]["GT_Total"]
    }
    for r in results_summary
])

print(scores_df.to_string(index=False))
print("\n✅ Ready for presentation!")


=== EXERCISE 3b) PIPELINE EXECUTION & EVALUATION ===

Model 1: Customer Onboarding (5 Tasks)
✗ Pipeline failed: Component named promptbuilder not found in the pipeline.

📊 EVALUATION RESULTS:
  Ground Truth (5 tasks): ['Verify Identity', 'Perform KYC Check', 'Set up Account', 'Approve Application', 'Fund Account']
  Extracted Tasks (0): []
  Precision: 0.0000 | Recall: 0.0000
  TP: 0 | FP: 0 | FN: 5

Model 2: Complaint Resolution (6 Tasks)
✗ Pipeline failed: Component named promptbuilder not found in the pipeline.

📊 EVALUATION RESULTS:
  Ground Truth (6 tasks): ['Log Complaint', 'Investigate Issue', 'Process Refund', 'Notify Finance', 'Send Resolution Email', 'Update CRM']
  Extracted Tasks (0): []
  Precision: 0.0000 | Recall: 0.0000
  TP: 0 | FP: 0 | FN: 6


📈 PRESENTATION TABLE - Exercise 3b Results
                         Model  Precision  Recall  TP  FP  FN  GT_Total
 Customer Onboarding (5 Tasks)        0.0     0.0   0   0   5         5
Complaint Resolution (6 Tasks)        0.0